In [22]:
import os
import ExtractMsg

docs=[]
docsQuote=[]
docsQuoteSubject=[]
docsNoQuoteSubject=[]
testDocSubject=[]
docsNoQuote =[]
testDoc =[]
pathQuote = 'C:/Users/StefanLap/Desktop/Bachelorarbeit/TrainQuote/'
pathNoQuote = 'C:/Users/StefanLap/Desktop/Bachelorarbeit/TrainNoQuote/'
pathTest = 'C:/Users/StefanLap/Desktop/Bachelorarbeit/TestPredict/'

    


def filldoclist(filepath, fillList, subjectList) :
        for file in os.listdir(filepath):
            #mail = open(filepath + file, 'r',encoding='utf8')
            mail = ExtractMsg.Message(filepath+file)
            try:
                
                fillList.append(mail.body)
                subjectList.append(file)
                continue
            except UnicodeDecodeError:
                continue                
            
            
filldoclist(pathQuote, docsQuote, docsQuoteSubject)
filldoclist(pathNoQuote, docsNoQuote, docsNoQuoteSubject)
filldoclist(pathTest, testDoc, testDocSubject)

#print (len(docsQuote))
#print (len(docsNoQuote))
#print (len(docsQuoteSubject))
        
docs = docsQuote + docsNoQuote
trainSubjects = docsQuoteSubject + docsNoQuoteSubject

print (len(testDoc))
print (len(testDocSubject))
    

25
25


In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

import numpy as np
import pandas as pd
from scipy.sparse import hstack


targetlist=[]

for i in range(0,len(docsQuote)):
    targetlist.append(1)

for i in range(0,len(docsNoQuote)):
    targetlist.append(0)


train_y = np.asarray(targetlist)







#training und test content im Block da Tfidftransformer bei beiden gebraucht werden
#def create_content_tfidf():
count_vect = CountVectorizer()
content_count = count_vect.fit_transform(docs)
tf_transformer = TfidfTransformer(use_idf=False).fit(content_count)
content_tfidf = tf_transformer.transform(content_count).toarray()


test_content_counts = count_vect.transform(testDoc)
test_content_tfidf = tf_transformer.transform(test_content_counts).toarray()


#training und test subject im Block da Tfidftransformer bei beiden gebraucht wird
#def create_subject_tfidf():
count_vect_subject = CountVectorizer()
subject_count = count_vect_subject.fit_transform(trainSubjects)
tf_subject_transformer =TfidfTransformer(use_idf=False).fit(subject_count)
subject_tfidf = tf_subject_transformer.transform(subject_count).toarray()

test_subject_counts = count_vect_subject.transform(testDocSubject)
test_subject_tfidf = tf_subject_transformer.transform(test_subject_counts).toarray()

#verusch tfidf des Inhalts und Countvectorizer des Betreffs zu kombinieren




print(content_tfidf.shape)
print(test_content_tfidf.shape)
print(subject_tfidf.shape)
print(test_subject_tfidf.shape)





numpyTarget = np.asarray([targetlist])

test_content_subject = np.hstack((test_content_tfidf, test_subject_tfidf))

      
#content_featureset = np.concatenate( (content_tfidf.toarray(), numpyTarget.T), axis=1)

x_test =  np.asarray(test_content_subject)
print(x_test.shape)
train_content_subject_set = np.hstack((content_tfidf, subject_tfidf))

train_content_subject_label_set =np.concatenate((train_content_subject_set, numpyTarget.T), axis=1)
#print(train_content_subject_label_set.shape)



(214, 7958)
(25, 7958)
(214, 440)
(25, 440)
(25, 8398)


In [42]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB

def fit_neuronal(training_dataset, targetlist):
    temp_neuronal = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
    temp_neuronal.fit(training_dataset, targetlist)
    return temp_neuronal

def fit_svm(training_dataset, targetlist):
    scaler = StandardScaler()
    X = scaler.fit_transform(training_dataset)
    temp_svm = SVC(C=10)
    temp_svm.fit(X, targetlist)
    return temp_svm


def fit_knn(training_dataset, targetlist):
    temp_knn= KNeighborsClassifier(n_neighbors=7)
    temp_knn.fit(training_dataset, targetlist)
    return temp_knn
def fit_NB(training_dataset, targetlist, bernoulli):
    if (bernoulli == True) :
        temp_NB = BernoulliNB()
        temp_NB.fit(training_dataset, targetlist)
        return temp_NB
    else :
        temp_NB = GaussianNB()
        temp_NB.fit(training_dataset, targetlist)
        return temp_NB
    
nb = fit_NB(train_content_subject_set, targetlist, False)
nb_B = fit_NB(train_content_subject_set, targetlist, True)
knn = fit_knn(train_content_subject_set, targetlist)
#svm = fit_svm(train_content_subject_set, targetlist)
#neuronal = fit_neuronal(train_content_subject_set, targetlist)




In [43]:
predicted_knn = knn.predict(x_test)
predicted_nb = nb.predict(x_test)
predicted_nb_B = nb_B.predict(x_test)
#predicted_svm = svm.predict(x_test)
#predicted_neuronal = neuronal.predict(x_test)
print(predicted_nb)
print(predicted_knn)
print(predicted_nb_B)
#print(predicted_svm)
#print(predicted_neuronal)

[1 0 0 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 0 0 1]
[1 0 0 1 0 0 1 1 1 0 1 0 1 0 0 1 1 1 1 1 0 1 1 0 0]
[0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 1]


In [48]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
ss = ShuffleSplit(n_splits=10, test_size=0.25, random_state=0)


In [52]:
scoring = 'accuracy'
score_knn= cross_val_score(knn, train_content_subject_set, targetlist, cv=ss, n_jobs=1, scoring=scoring)
score_nb_B= cross_val_score(nb_B, train_content_subject_set, targetlist, cv=ss, n_jobs=1, scoring=scoring)
score_nb= cross_val_score(nb, train_content_subject_set, targetlist, cv=ss, n_jobs=1, scoring=scoring)
#score_svm= cross_val_score(svm, train_content_subject_set, targetlist, cv=k_fold, n_jobs=1, scoring=scoring)
#score_neuronal= cross_val_score(neuronal, train_content_subject_set, targetlist, cv=k_fold, n_jobs=1, scoring=scoring)
print(score_knn)
print(score_nb)
print(score_nb_B)
#print(score_svm)
#print(score_neuronal)
#print("Accuracy: %0.2f (+/- %0.2f)" % (score_svm.mean(), scores.std() * 2))

[ 0.83333333  0.7962963   0.74074074  0.74074074  0.74074074  0.75925926
  0.92592593  0.72222222  0.81481481  0.81481481]
[ 0.81481481  0.85185185  0.74074074  0.90740741  0.75925926  0.88888889
  0.85185185  0.7962963   0.81481481  0.81481481]
[ 0.77777778  0.83333333  0.72222222  0.88888889  0.85185185  0.81481481
  0.81481481  0.87037037  0.81481481  0.7962963 ]


In [53]:
print(round(np.mean(score_knn)*100, 2))
print(round(np.mean(score_nb_B)*100, 2))
print(round(np.mean(score_nb)*100, 2))
#print(round(np.mean(score_svm)*100, 2))
#print(round(np.mean(score_neuronal)*100, 2))

78.89
81.85
82.41


In [28]:
from sklearn.metrics import average_precision_score
y_test= [1,0,0]
print(x_test.shape)
#y_score = knn.descision_function(x_test,y_test)
#average_precision = average_precision_score()

(25, 8398)
